In [988]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
import warnings

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
%matplotlib inline
warnings.filterwarnings('ignore')


In [989]:
#Load datasets
df_train=pd.read_csv("https://datahack-prod.s3.amazonaws.com/train_file/train_v9rqX0R.csv")
df_train

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [990]:
df_train.shape

(8523, 12)

In [991]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [992]:
df_train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [993]:
#Load datasets
df_test=pd.read_csv("https://datahack-prod.s3.amazonaws.com/test_file/test_AbJTz2l.csv")
df_test

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3
...,...,...,...,...,...,...,...,...,...,...,...
5676,FDB58,10.500,Regular,0.013496,Snack Foods,141.3154,OUT046,1997,Small,Tier 1,Supermarket Type1
5677,FDD47,7.600,Regular,0.142991,Starchy Foods,169.1448,OUT018,2009,Medium,Tier 3,Supermarket Type2
5678,NCO17,10.000,Low Fat,0.073529,Health and Hygiene,118.7440,OUT045,2002,NaN,Tier 2,Supermarket Type1
5679,FDJ26,15.300,Regular,0.000000,Canned,214.6218,OUT017,2007,NaN,Tier 2,Supermarket Type1


In [994]:
df_test.shape

(5681, 11)

In [995]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            5681 non-null   object 
 1   Item_Weight                4705 non-null   float64
 2   Item_Fat_Content           5681 non-null   object 
 3   Item_Visibility            5681 non-null   float64
 4   Item_Type                  5681 non-null   object 
 5   Item_MRP                   5681 non-null   float64
 6   Outlet_Identifier          5681 non-null   object 
 7   Outlet_Establishment_Year  5681 non-null   int64  
 8   Outlet_Size                4075 non-null   object 
 9   Outlet_Location_Type       5681 non-null   object 
 10  Outlet_Type                5681 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 488.3+ KB


In [996]:
df_test.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
count,4705.000000,5681.000000,5681.000000,5681.000000
mean,12.695633,0.065684,141.023273,1997.828903
std,4.664849,0.051252,61.809091,8.372256
min,4.555000,0.000000,31.990000,1985.000000
25%,8.645000,0.027047,94.412000,1987.000000
50%,12.500000,0.054154,141.415400,1999.000000
75%,16.700000,0.093463,186.026600,2004.000000
max,21.350000,0.323637,266.588400,2009.000000


In [997]:
# Display vars that have 1 or more missing data points
print('Training Data:')
print(df_train.isnull().sum().sort_values(ascending=False).head(20))# display top categories with missing data
print('---------------------------')
print('Test Data:')
print(df_test.isnull().sum().sort_values(ascending=False).head(35)) # check in Test data too so you don't build model around vars that don't exist

Training Data:
Outlet_Size                  2410
Item_Weight                  1463
Item_Outlet_Sales               0
Outlet_Type                     0
Outlet_Location_Type            0
Outlet_Establishment_Year       0
Outlet_Identifier               0
Item_MRP                        0
Item_Type                       0
Item_Visibility                 0
Item_Fat_Content                0
Item_Identifier                 0
dtype: int64
---------------------------
Test Data:
Outlet_Size                  1606
Item_Weight                   976
Outlet_Type                     0
Outlet_Location_Type            0
Outlet_Establishment_Year       0
Outlet_Identifier               0
Item_MRP                        0
Item_Type                       0
Item_Visibility                 0
Item_Fat_Content                0
Item_Identifier                 0
dtype: int64


In [998]:
# fill up missing values with mode
df_train['Outlet_Size'] = df_train['Outlet_Size'].fillna(df_train['Outlet_Size'].mode()[0])
df_test['Outlet_Size'] = df_test['Outlet_Size'].fillna(df_test['Outlet_Size'].mode()[0])

df_train['Item_Weight'] = df_train['Item_Weight'].fillna(df_train['Item_Weight'].mode()[0])
df_test['Item_Weight'] = df_test['Item_Weight'].fillna(df_test['Item_Weight'].mode()[0])

In [999]:
df_train.isnull().sum().sum(), df_test.isnull().sum().sum()

(0, 0)

In [1000]:
weight=df_train.groupby('Item_Weight')['Item_Outlet_Sales'].sum().reset_index()
weight

,Item_Weight,Item_Outlet_Sales
0,4.555,7046.8272
1,4.590,6338.4160
2,4.610,13955.1680
3,4.615,16312.1000
4,4.635,10922.4490
...,...,...
410,21.000,7168.6686
411,21.100,31475.0292
412,21.200,13818.6790
413,21.250,52780.6292


In [1001]:
df_train.Item_Weight.isnull().sum().sum(), df_test.Item_Weight.isnull().sum().sum()

(0, 0)

In [1002]:
fat=df_train.groupby('Item_Fat_Content')['Item_Outlet_Sales'].sum().reset_index()
fat

,Item_Fat_Content,Item_Outlet_Sales
0,LF,6.552424e+05
1,Low Fat,1.101503e+07
2,Regular,6.457454e+06
3,low fat,2.338270e+05
4,reg,2.295765e+05


In [1003]:
fat1={'LF':0, 'Low Fat':0, 'low fat':0,'Regular':1,'reg':1 }
df_train.Item_Fat_Content=df_train.Item_Fat_Content.map(fat1)
df_test.Item_Fat_Content=df_test.Item_Fat_Content.map(fat1)

In [1004]:
fat2=df_train.groupby('Item_Fat_Content')['Item_Outlet_Sales'].sum().reset_index()
fat2

,Item_Fat_Content,Item_Outlet_Sales
0,0,1.190409e+07
1,1,6.687031e+06


In [1005]:
df_train.Item_Fat_Content.isnull().sum().sum(), df_test.Item_Fat_Content.isnull().sum().sum()

(0, 0)

In [1006]:
vis=df_train.groupby('Item_Visibility')['Item_Outlet_Sales'].sum().reset_index()
vis

,Item_Visibility,Item_Outlet_Sales
0,0.000000,1.169058e+06
1,0.003575,3.229796e+03
2,0.003589,1.691798e+03
3,0.003598,2.922196e+03
4,0.003599,3.075996e+03
...,...,...
7875,0.309390,1.384864e+02
7876,0.311090,1.544656e+02
7877,0.321115,1.997400e+02
7878,0.325781,7.610094e+02


In [1007]:
df_train.Item_Visibility.isnull().sum().sum(), df_test.Item_Visibility.isnull().sum().sum()

(0, 0)

In [1008]:
type1=df_train.groupby('Item_Type')['Item_Outlet_Sales'].sum().reset_index()
type1

,Item_Type,Item_Outlet_Sales
0,Baking Goods,1.265525e+06
1,Breads,5.532372e+05
2,Breakfast,2.322990e+05
3,Canned,1.444151e+06
4,Dairy,1.522594e+06
5,Frozen Foods,1.825735e+06
6,Fruits and Vegetables,2.820060e+06
7,Hard Drinks,4.577934e+05
8,Health and Hygiene,1.045200e+06
9,Household,2.055494e+06


In [1009]:
type2={'Baking Goods':1, 'Breads':2, 'Breakfast':3,'Canned':4,'Dairy':5, 'Frozen Foods':6, 'Fruits and Vegetables':7,
       'Hard Drinks':8, 'Health and Hygiene':9, 'Household': 10, 'Meat':11, 'Others':12, 'Seafood':13, 'Snack Foods':14,
       'Soft Drinks':15, 'Starchy Foods':16}
df_train.Item_Type=df_train.Item_Type.map(type2)
df_test.Item_Type=df_test.Item_Type.map(type2)

In [1010]:
type3=df_train.groupby('Item_Type')['Item_Outlet_Sales'].sum().reset_index()
type3

,Item_Type,Item_Outlet_Sales
0,1,1.265525e+06
1,2,5.532372e+05
2,3,2.322990e+05
3,4,1.444151e+06
4,5,1.522594e+06
5,6,1.825735e+06
6,7,2.820060e+06
7,8,4.577934e+05
8,9,1.045200e+06
9,10,2.055494e+06


In [1011]:
df_train.Item_Type.isnull().sum().sum(), df_test.Item_Type.isnull().sum().sum()

(0, 0)

In [1012]:
mrp=df_train.groupby('Item_MRP')['Item_Outlet_Sales'].sum().reset_index()
mrp

,Item_MRP,Item_Outlet_Sales
0,31.2900,898.8300
1,31.4900,466.0600
2,31.8900,366.1900
3,31.9558,1358.2320
4,32.0558,1018.6740
...,...,...
5933,266.1884,8479.6288
5934,266.2884,2914.8724
5935,266.5884,7949.6520
5936,266.6884,4239.8144


In [1013]:
df_train.Item_MRP.isnull().sum().sum(), df_test.Item_MRP.isnull().sum().sum()

(0, 0)

In [1014]:
df_train['Outlet_Identifier'] = df_train['Outlet_Identifier'].str.lstrip("OUT0")
df_test['Outlet_Identifier'] = df_test['Outlet_Identifier'].str.lstrip("OUT0")

In [1015]:
ident=df_train.groupby('Outlet_Identifier')['Item_Outlet_Sales'].sum().reset_index()
ident

,Outlet_Identifier,Item_Outlet_Sales
0,10,1.883402e+05
1,13,2.142664e+06
2,17,2.167465e+06
3,18,1.851823e+06
4,19,1.796941e+05
5,27,3.453926e+06
6,35,2.268123e+06
7,45,2.036725e+06
8,46,2.118395e+06
9,49,2.183970e+06


In [1016]:
df_train.Outlet_Identifier.isnull().sum().sum(), df_test.Outlet_Identifier.isnull().sum().sum()

(0, 0)

In [1017]:
year=df_train.groupby('Outlet_Establishment_Year')['Item_Outlet_Sales'].sum().reset_index()
year

,Outlet_Establishment_Year,Item_Outlet_Sales
0,1985,3.633620e+06
1,1987,2.142664e+06
2,1997,2.118395e+06
3,1998,1.883402e+05
4,1999,2.183970e+06
5,2002,2.036725e+06
6,2004,2.268123e+06
7,2007,2.167465e+06
8,2009,1.851823e+06


In [1018]:
df_train.Outlet_Establishment_Year.isnull().sum().sum(), df_test.Outlet_Establishment_Year.isnull().sum().sum()

(0, 0)

In [1019]:
size=df_train.groupby('Outlet_Size')['Item_Outlet_Sales'].sum().reset_index()
size

,Outlet_Size,Item_Outlet_Sales
0,High,2.142664e+06
1,Medium,1.188225e+07
2,Small,4.566212e+06


In [1020]:
size1={'High':1, 'Medium':2, 'Small':3}
df_train.Outlet_Size=df_train.Outlet_Size.map(size1)
df_test.Outlet_Size=df_test.Outlet_Size.map(size1)

In [1021]:
size2=df_train.groupby('Outlet_Size')['Item_Outlet_Sales'].sum().reset_index()
size2

,Outlet_Size,Item_Outlet_Sales
0,1,2.142664e+06
1,2,1.188225e+07
2,3,4.566212e+06


In [1022]:
df_train.Outlet_Size.isnull().sum().sum(), df_test.Outlet_Size.isnull().sum().sum()

(0, 0)

In [1023]:
df_train['Outlet_Location_Type'] = df_train['Outlet_Location_Type'].str.lstrip("Tier ")
df_test['Outlet_Location_Type'] = df_test['Outlet_Location_Type'].str.lstrip("Tier ")

In [1024]:
locale=df_train.groupby('Outlet_Location_Type')['Item_Outlet_Sales'].sum().reset_index()
locale

,Outlet_Location_Type,Item_Outlet_Sales
0,1,4.482059e+06
1,2,6.472314e+06
2,3,7.636753e+06


In [1025]:
df_train.Outlet_Location_Type.isnull().sum().sum(), df_test.Outlet_Location_Type.isnull().sum().sum()

(0, 0)

In [1026]:
df_train['Outlet_Type'] = df_train['Outlet_Type'].str.lstrip("Supermarket Type")
df_test['Outlet_Type'] = df_test['Outlet_Type'].str.lstrip("Supermarket Type")

In [1027]:
type0=df_train.groupby('Outlet_Type')['Item_Outlet_Sales'].sum().reset_index()
type0

,Outlet_Type,Item_Outlet_Sales
0,1,1.291734e+07
1,2,1.851823e+06
2,3,3.453926e+06
3,Grocery Store,3.680343e+05


In [1028]:
type3={'1':1, '2':2, '3':3, 'Grocery Store':4}
df_train.Outlet_Type=df_train.Outlet_Type.map(type3)
df_test.Outlet_Type=df_test.Outlet_Type.map(type3)

In [1029]:
type3=df_train.groupby('Outlet_Type')['Item_Outlet_Sales'].sum().reset_index()
type3

,Outlet_Type,Item_Outlet_Sales
0,1,1.291734e+07
1,2,1.851823e+06
2,3,3.453926e+06
3,4,3.680343e+05


In [1030]:
df_train.Outlet_Type.isnull().sum().sum(), df_test.Outlet_Type.isnull().sum().sum()

(0, 0)

In [1031]:
df_train

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,0,0.016047,5,249.8092,49,1999,2,1,1,3735.1380
1,DRC01,5.920,1,0.019278,15,48.2692,18,2009,2,3,2,443.4228
2,FDN15,17.500,0,0.016760,11,141.6180,49,1999,2,1,1,2097.2700
3,FDX07,19.200,1,0.000000,7,182.0950,10,1998,2,3,4,732.3800
4,NCD19,8.930,0,0.000000,10,53.8614,13,1987,1,3,1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,0,0.056783,14,214.5218,13,1987,1,3,1,2778.3834
8519,FDS36,8.380,1,0.046982,1,108.1570,45,2002,2,2,1,549.2850
8520,NCJ29,10.600,0,0.035186,9,85.1224,35,2004,3,2,1,1193.1136
8521,FDN46,7.210,1,0.145221,14,103.1332,18,2009,2,3,2,1845.5976


In [1032]:
df_test

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,0,0.007565,14,107.8622,49,1999,2,1,1
1,FDW14,8.300,1,0.038428,5,87.3198,17,2007,2,2,1
2,NCN55,14.600,0,0.099575,12,241.7538,10,1998,2,3,4
3,FDQ58,7.315,0,0.015388,14,155.0340,17,2007,2,2,1
4,FDY38,10.500,1,0.118599,5,234.2300,27,1985,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...
5676,FDB58,10.500,1,0.013496,14,141.3154,46,1997,3,1,1
5677,FDD47,7.600,1,0.142991,16,169.1448,18,2009,2,3,2
5678,NCO17,10.000,0,0.073529,9,118.7440,45,2002,2,2,1
5679,FDJ26,15.300,1,0.000000,4,214.6218,17,2007,2,2,1


In [1033]:
#define x, y and x_test
y = df_train["Item_Outlet_Sales"]
features = ["Item_Weight", "Item_Fat_Content", "Item_Visibility", "Item_Type", "Item_MRP", "Outlet_Identifier", "Outlet_Establishment_Year", "Outlet_Size", "Outlet_Location_Type"]
X = pd.get_dummies(df_train[features])
X_test = pd.get_dummies(df_test[features])

In [1034]:
# define standard scaler
scaler = StandardScaler()
# transform data
X= scaler.fit_transform(X)
X

array([[-0.81151126, -0.73814723, -0.97073217, ...,  1.60283982,
        -0.69667837, -0.8047318 ],
       [-1.60975569,  1.35474328, -0.90811123, ..., -0.62389266,
        -0.69667837,  1.24265003],
       [ 1.12505806, -0.73814723, -0.95691733, ...,  1.60283982,
        -0.69667837, -0.8047318 ],
       ...,
       [-0.50449417, -0.73814723, -0.59978449, ..., -0.62389266,
         1.43538258, -0.8047318 ],
       [-1.30510027,  1.35474328,  1.53287976, ..., -0.62389266,
        -0.69667837,  1.24265003],
       [ 0.48740719, -0.73814723, -0.41193591, ...,  1.60283982,
        -0.69667837, -0.8047318 ]])

In [1035]:
X_test= scaler.fit_transform(X_test)
X_test

array([[ 1.94956229, -0.74081066, -1.13407715, ...,  1.60264386,
        -0.69658363, -0.80474953],
       [-0.92914563,  1.34987258, -0.53185019, ..., -0.62396895,
         1.4355778 , -0.80474953],
       [ 0.52754995, -0.74081066,  0.6613164 , ..., -0.62396895,
        -0.69658363,  1.24262265],
       ...,
       [-0.53606904, -0.74081066,  0.15307374, ..., -0.62396895,
         1.4355778 , -0.80474953],
       [ 0.68940501,  1.34987258, -1.28168987, ..., -0.62396895,
         1.4355778 , -0.80474953],
       [-0.6516798 ,  1.34987258,  0.76171558, ..., -0.62396895,
         1.4355778 , -0.80474953]])

In [1036]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
test_X = test_data_imputed.copy()[all_features]

In [1037]:
# shape of training and validation set
(train_X.shape, val_X.shape), (train_y.shape, val_y.shape)

(((6392, 20), (2131, 20)), ((6392,), (2131,)))

In [1038]:
# checking the version of keras
import keras
print(keras.__version__)

2.4.3


In [1039]:
# checking the version of tensorflow
import tensorflow as tf
print(tf.__version__)

2.3.0


In [1040]:
# importing the sequential model
from keras.models import Sequential

In [1041]:
# importing different layers from keras
from keras.layers import InputLayer, Dense 

In [1042]:
# number of input neurons
train_X.shape

(6392, 20)

In [1043]:
# number of features in the data
train_X.shape[1]

20

In [1044]:
# defining input neurons
input_neurons = train_X.shape[1]

In [1045]:
# define number of output neurons
output_neurons = 1

In [1046]:
# define hidden layers and neuron in each layer
number_of_hidden_layers = 2
neuron_hidden_layer_1 = 10
neuron_hidden_layer_2 = 5

In [1047]:
# defining the architecture of the model
model = Sequential()
model.add(InputLayer(input_shape=(input_neurons,)))
model.add(Dense(units=neuron_hidden_layer_1, activation='relu'))
model.add(Dense(units=neuron_hidden_layer_2, activation='relu'))
model.add(Dense(units=output_neurons, activation='linear'))

In [1048]:
# summary of the model
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_46 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 6         
Total params: 271
Trainable params: 271
Non-trainable params: 0
_________________________________________________________________


In [1049]:
# number of parameters between input and first hidden layer
input_neurons*neuron_hidden_layer_1

200

In [1050]:
# number of parameters between input and first hidden layer
# adding the bias for each neuron of first hidden layer
input_neurons*neuron_hidden_layer_1 + 10

210

In [1051]:
# number of parameters between first and second hidden layer
neuron_hidden_layer_1*neuron_hidden_layer_2 + 5

55

In [1052]:
# number of parameters between second hidden and output layer
neuron_hidden_layer_2*output_neurons + 1

6

In [1053]:
# compiling the model
# loss as binary_crossentropy, since we have binary classification problem
# defining the optimizer as adam
# Evaluation metric as accuracy
model.compile(loss='mse',optimizer='Adam',metrics=['accuracy'])

In [1054]:
train_X

array([[-1.33107864, -0.73814723,  0.85479095, ..., -0.62389266,
         1.43538258, -0.8047318 ],
       [-0.03216019,  1.35474328,  0.15905739, ..., -0.62389266,
         1.43538258, -0.8047318 ],
       [-1.11380501, -0.73814723,  0.87775441, ..., -0.62389266,
         1.43538258, -0.8047318 ],
       ...,
       [ 1.20771651, -0.73814723, -0.00624063, ...,  1.60283982,
        -0.69667837, -0.8047318 ],
       [-0.74066117, -0.73814723, -0.32007616, ..., -0.62389266,
        -0.69667837,  1.24265003],
       [-0.13843534, -0.73814723, -0.43722179, ..., -0.62389266,
        -0.69667837,  1.24265003]])

In [1055]:
# training the model
# passing the independent and dependent features for training set for training the model
# validation data will be evaluated at the end of each epoch
# setting the epochs as 50
# storing the trained model in model_history variable which will be used to visualize the training process

train_X = np.asarray(train_X, dtype=np.float)
train_y = np.asarray(train_y, dtype=np.float)
val_X = np.asarray(val_X, dtype=np.float)
val_y = np.asarray(val_y, dtype=np.float)

model_history = model.fit(train_X, train_y, validation_data=(val_X, val_y), epochs=50)

Epoch 1/50
200/200 [==============================] - 0s 2ms/step - loss: 7686182.5000 - accuracy: 0.0000e+00 - val_loss: 7507966.0000 - val_accuracy: 0.0000e+00
Epoch 2/50
200/200 [==============================] - 0s 1ms/step - loss: 7547167.5000 - accuracy: 0.0000e+00 - val_loss: 7249019.0000 - val_accuracy: 0.0000e+00
Epoch 3/50
200/200 [==============================] - 0s 1ms/step - loss: 7105637.0000 - accuracy: 0.0000e+00 - val_loss: 6622539.5000 - val_accuracy: 0.0000e+00
Epoch 4/50
200/200 [==============================] - 0s 1ms/step - loss: 6268627.5000 - accuracy: 0.0000e+00 - val_loss: 5614554.5000 - val_accuracy: 0.0000e+00
Epoch 5/50
200/200 [==============================] - 0s 1ms/step - loss: 5100330.5000 - accuracy: 0.0000e+00 - val_loss: 4375584.0000 - val_accuracy: 0.0000e+00
Epoch 6/50
200/200 [==============================] - 0s 1ms/step - loss: 3830727.0000 - accuracy: 0.0000e+00 - val_loss: 3182723.7500 - val_accuracy: 0.0000e+00
Epoch 7/50
200/200 [========

In [1056]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
predictions = model.predict(X_test)
predictions

array([[1744.4048],
       [1433.2289],
       [ 883.6203],
       ...,
       [1696.6794],
       [3681.5393],
       [1177.3772]], dtype=float32)

In [1057]:
predictions

array([[1744.4048],
       [1433.2289],
       [ 883.6203],
       ...,
       [1696.6794],
       [3681.5393],
       [1177.3772]], dtype=float32)

In [1058]:
predictions.shape

(5681, 1)

In [1059]:
predictions.ndim

2

In [1060]:
predictions=predictions.flatten()
predictions.ndim

1

In [1061]:
#Load sample submission
df_sample=pd.read_csv("https://datahack-prod.s3.amazonaws.com/sample_submission/sample_submission_8RXa3c6.csv")
df_sample

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1000
1,FDW14,OUT017,1000
2,NCN55,OUT010,1000
3,FDQ58,OUT017,1000
4,FDY38,OUT027,1000
...,...,...,...
5676,FDB58,OUT046,1000
5677,FDD47,OUT018,1000
5678,NCO17,OUT045,1000
5679,FDJ26,OUT017,1000


In [1062]:
df_test.Outlet_Identifier="OUT0"+df_test.Outlet_Identifier

In [1063]:
output = pd.DataFrame({'Item_Identifier': df_test.Item_Identifier,'Outlet_Identifier': df_test.Outlet_Identifier,
                       'Item_Outlet_Sales': predictions})
output.to_csv('submission.csv', index=False)
output

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1744.404785
1,FDW14,OUT017,1433.228882
2,NCN55,OUT010,883.620300
3,FDQ58,OUT017,2588.942627
4,FDY38,OUT027,5803.246582
...,...,...,...
5676,FDB58,OUT046,2264.766113
5677,FDD47,OUT018,2395.046387
5678,NCO17,OUT045,1696.679443
5679,FDJ26,OUT017,3681.539307
